In [ ]:
# PySpark + MinIO demo notebook

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# 1. Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Spark MinIO Demo") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

# 2. Cấu hình kết nối MinIO (S3 API)
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "admin")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "admin123")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")

input_path = "s3a://demo-bucket/input/sample.csv"
output_path = "s3a://demo-bucket/output/result.csv"

# 3. Đọc dữ liệu từ MinIO
df = spark.read.option("header", "true").csv(input_path)
print("Input Data:")
df.show()

# 4. Transform dữ liệu (ví dụ: chọn cột, đổi tên, tính toán)
df_transformed = df.select(
    col("id"),
    col("value").cast("double"),
    (col("value").cast("double") * 2).alias("value_double")
)

# 5. Ghi kết quả về MinIO
df_transformed.write.mode("overwrite").option("header", "true").csv(output_path)
print(f"Output saved to {output_path}")
